In [1]:
import anndata as ad
import numpy as np
import pandas as pd
import os
import gzip
import urllib

`DATA_DIR`: Where the data should exist or be downloaded to if not. 

`METADATA_FILE`: .txt file associated with the paper. 

In [2]:
DATA_DIR = '.'
METADATA_FILE = './PRJEB3291.txt'

The next cell loads the metadata file into a Pandas dataframe. 

In [3]:
with open(METADATA_FILE, 'r') as f:
    meta_df = pd.read_csv(f, delimiter='\t')
meta_df

,study_accession,sample_accession,experiment_accession,run_accession,tax_id,scientific_name,fastq_ftp,submitted_ftp,sra_ftp
0,PRJEB3291,SAMEA1695337,ERX157273,ERR181163,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/ERR181163/...,ftp.sra.ebi.ac.uk/vol1/run/ERR181/ERR181163/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR181/ERR181163
1,PRJEB3291,SAMEA1695345,ERX157281,ERR181171,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/ERR181171/...,ftp.sra.ebi.ac.uk/vol1/run/ERR181/ERR181171/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR181/ERR181171
2,PRJEB3291,SAMEA1695334,ERX157282,ERR181172,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/ERR181172/...,ftp.sra.ebi.ac.uk/vol1/run/ERR181/ERR181172/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR181/ERR181172
3,PRJEB3291,SAMEA1695342,ERX157408,ERR181298,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/ERR181298/...,ftp.sra.ebi.ac.uk/vol1/run/ERR181/ERR181298/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR181/ERR181298
4,PRJEB3291,SAMEA1695346,ERX159245,ERR183498,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR183/ERR183498/...,ftp.sra.ebi.ac.uk/vol1/run/ERR183/ERR183498/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR183/ERR183498
5,PRJEB3291,SAMEA1695346,ERX159246,ERR183499,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR183/ERR183499/...,ftp.sra.ebi.ac.uk/vol1/run/ERR183/ERR183499/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR183/ERR183499
6,PRJEB3291,SAMEA1695347,ERX159247,ERR183500,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR183/ERR183500/...,ftp.sra.ebi.ac.uk/vol1/run/ERR183/ERR183500/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR183/ERR183500
7,PRJEB3291,SAMEA1695347,ERX159248,ERR183501,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR183/ERR183501/...,ftp.sra.ebi.ac.uk/vol1/run/ERR183/ERR183501/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR183/ERR183501
8,PRJEB3291,SAMEA1695347,ERX159249,ERR183502,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR183/ERR183502/...,ftp.sra.ebi.ac.uk/vol1/run/ERR183/ERR183502/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR183/ERR183502
9,PRJEB3291,SAMEA1695340,ERX159250,ERR183503,9606,Homo sapiens,ftp.sra.ebi.ac.uk/vol1/fastq/ERR183/ERR183503/...,ftp.sra.ebi.ac.uk/vol1/run/ERR183/ERR183503/Lo...,ftp.sra.ebi.ac.uk/vol1/err/ERR183/ERR183503


Testing with a single file for demonstration. 

Remove the `meta_df` sublist index in the first `for` loop to test with all files.

In [5]:
# controls how many reads to load from the file (does not change download size)
n_sample = 1000

for url in meta_df['submitted_ftp'][:1]: # remove sublist index to download all missing files
    filename = url.split('/')[-1]
    if filename not in os.listdir(DATA_DIR):
        print('Downloading from', url)
        urllib.request.urlretrieve(f'ftp://{url}', f'{DATA_DIR}/{filename}')
        print('Done.')

    seqlen = set()
    reads = []

    # get every fourth line starting with index 1 (every second line in four line group)
    for i, r in enumerate(gzip.open(f'{DATA_DIR}/{filename}')):
        if i  % 4 == 1:
            s = r.strip().decode("utf-8")
            seqlen.add(len(s))
            reads.append(s)
            if n_sample is not None and len(reads) >= n_sample:
                break
    df = pd.DataFrame(reads, columns=["seq"])

    # only get those with the longest length
    df = df[df["seq"].str.len() == max(seqlen)]

    # occurance counts of each sequence
    df = df.seq.value_counts().reset_index()
    df.columns = ["seq", "counts"]

    print(df)

                                   seq  counts
0    ACAGATCGGAAGAGCTCGTATGCCGTCTTCTGC       3
1    AAAAGGTAAAGCAGAAGGAAGAGGAGCAGAGTT       2
2    AGCTTGCTGACCAGTGAGCTTATTTTTGTGGAA       2
3    TCTCAGCCACCCGTGTGGGCATTGCTGTTTGGT       1
4    GGATGATTTGCTCCAAGCATAAAAGAGAATTGT       1
..                                 ...     ...
991  TGGTGATCGGCGAGCTTGGCGTGGGCAAGACCA       1
992  GCACTTTATTGCCTTGGGGGTTATAAGTGAACT       1
993  GGAAAATGCGCATCACATAACTATGATTCTTTC       1
994  GTCCAACATGGTGAAACCCCATCTCTACTAAAA       1
995  TCCAAAGAGGTTCACAGCGCCGAGTCTGCATGA       1

[996 rows x 2 columns]


In [7]:
# construct data with sequences as observations and counts as variables
# reshape because array we pass to anndata has to be 2d (alternative is to add a dummy index column)
adata = ad.AnnData(df['counts'].to_numpy().reshape(-1, 1)) 
adata.obs_names = df['seq'].to_numpy()
adata.var_names = ['count']

# add paper information to uns.
adata.uns['author'] = 'Jolma'
adata.uns['doi'] = '10.1016/j.cell.2012.12.009'
adata.uns['year'] = 2013
adata.uns['organism'] = 'homo sapiens'

adata

/var/folders/17/ksjp4vcn2gs552rnysdxktrc0000gn/T/ipykernel_41332/1432522977.py:3: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(df['counts'].to_numpy().reshape(-1, 1))


AnnData object with n_obs × n_vars = 996 × 1
    uns: 'author', 'doi', 'year', 'organism'

In [12]:
print('accessing by sequence: ', adata['ACAGATCGGAAGAGCTCGTATGCCGTCTTCTGC'])
print('accessing counts: ', adata['ACAGATCGGAAGAGCTCGTATGCCGTCTTCTGC'].X)

accessing by sequence:  View of AnnData object with n_obs × n_vars = 1 × 1
    uns: 'author', 'doi', 'year', 'organism'
accessing counts:  [[3.]]
